# Overview

This example will open an XNAT connection using the `xnat_wrapper.Connector` class, find studies using a list of study UIDs, download those studies from PACS into an XNAT project, convert the XNAT file structure to BIDS format, run the XNAT MRIQC docker container on the project, and finally save the MRIQC results to a CSV file.

## Requirements

The only external package used by `xnat_wrapper` is [pyxnat](https://pyxnat.github.io/pyxnat). Use `pip` to install this package:

`python3 -m pip install pyxnat`

## Import Packages

Add the relative location of the `xnat_wrapper` directory to your path (in this case, the parent directory):

In [ ]:
import os
import sys
cwd = os.getcwd()
sys.path.append(cwd + '/../')

Import the XNAT wrapper and utilities:

In [ ]:
from xnat_wrapper import (
    Connector,
    format_err,
    json_to_csv
)

## Define Variables

The constants/variables we will need throughout this script are stored in `constants.py`. This file currently has the following variables defined:

- ***xnat_ip***: The IP address of the XNAT server
- ***xnat_username***: The username used to login to the XNAT server
- ***xnat_password***: The password used to login to the XNAT server
- ***xnat_project***: The name of the XNAT project on the server
- ***results_file***: Name of the JSON file where the results will be stored
- ***results_csv***: Name of the CSV file where the results will be stored (converted from JSON)
- ***uid_list***: List of study UIDs that we will import from PACS
- ***filters***: List of filters that we will use to only pull relevant files
- ***commands***: List of commands (XNAT plugins/containers) that we will run on the imported files
- ***resource_dir***: XNAT resource folder in which to look for the appropriate JSON files
- ***add_keys***: Additional headers in CSV that are not base key/value pairs in the json structures (files) returned from XNAT

In [ ]:
from constants import *

The `xnat_wrapper.Connector` class that we just imported will be how we connect and access the XNAT instance. The `format_err` and `json_to_csv` are utilities we will use later.

## Connect to XNAT

Login to XNAT session using config file (JSON file containing XNAT server IP address, username, and password). As an alternative, you can log into the session by passing your server, username, and password (which is what is in the config file) directly.

In [ ]:
xnat = Connector(server=xnat_ip,
                user=xnat_username,
                password=xnat_password,
                project=xnat_project)

Check that we are successfully connected to the XNAT server:

In [ ]:
if xnat.is_connected():
    print('Successfully connected to the XNAT server!')
else:
    print('Could not establish a connection to the XNAT server.')
    sys.exit()

## Find and Import Studies

After we successfully connect to the XNAT server, we will want to import some studies to our project. The `xnat_wrapper.Connector` class has an `importer` utility that we will use to do this. 

First, set the study UIDs (***uid_list***) using the `importer.set_uids()` function:

In [ ]:
xnat.importer.set_uids(uid_list)

We also want to make sure that we only find the studies relevant to our project, so we can set a study filter (***filters***) using the `importer.set_filters()` function:

In [ ]:
xnat.importer.set_filters(filters)

With the UIDs and filters set, we can now find all relevant studies using the `importer.find_studies()` function. All relevant studies found will be stored in `xnat.importer.studies`.

In [ ]:
xnat.importer.find_studies()

Once we find all of the studies, we need to import them into our project using the `importer.import_studies()` function. This function will return `True` if the import request was successful (not if the studies were imported) and `False` if the request failed. If the request is successful, we can monitor the status of the import using the `importer.monitor_import_queue()` function. If this fails, make sure to close the XNAT connection prior to exiting the script using the `xnat.close_session()` function.

In [ ]:
if xnat.importer.import_studies():
    xnat.importer.monitor_import_queue(timeout=180, time_refresh=True)
else:
    print('Unable to import studies to {}.'.format(xnat_project))
    xnat.close_session()
    sys.exit()

## Run XNAT Commands on a Project

After we successfully import the studies to our project, now we want to run a series of commands (***commands***) on the project files. All command-related functions use the `commands` utility of the `xnat_wrapper.Connector` (in this example it is accessed via `xnat.commands`). 

First, we need to tell the utility which commands to use. These commands can be added via `commands.set_commands()` or explicitly called when using the `commands.run_commands()` function.

In [ ]:
xnat.commands.set_commands(commands)

Next, we need to find all experiments in the project so that the commands know which files to access during runtime:

In [ ]:
xnat.commands.find_project_experiments()

Check that some sessions/scans were found before moving on. Alternatively, a dictionary containing sessions and scans can be retrieved using the `commands.get_project_experiments()` function.

In [ ]:
if not xnat.commands.has_experiments(): 
    print('No scans or sessions found in {}.'.format(xnat_project))
    xnat.close_session()
    sys.exit()

If scans and/or sessions were found, run our list of commands on them using the `commands.run_commands()` function:

In [ ]:
xnat.commands.run_commands()

Write results to file (***results_file***). This function will check that results exist prior to file creation. The `.json` extension will be added if not supplied.

In [ ]:
xnat.commands.save_results(results_file)

The last command in our list was the MRIQC command. If the MRIQC function was successful (i.e. if there are results available), there should now be JSON files in the project. Those need to be combined into a single CSV file so the results can be plotted later. 

In [ ]:
if xnat.commands.get_results():
    json_files = xnat.commands.download_json_files(resource=resource_dir)
    json_to_csv(json_files,results_csv,keys=add_keys)

Once that is complete, we are finished! All that is left to do is close out the XNAT session.

In [ ]:
xnat.close_session()